推論するよ
すごいよ

In [1]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

Looking in links: /kaggle/input/lmsys-wheel-files
Processing /kaggle/input/lmsys-wheel-files/peft-0.11.1-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl


In [2]:
import json
import unicodedata

import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig, AutoTokenizer
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

2024-07-29 02:10:37.181742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 02:10:37.181872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 02:10:37.359533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
assert torch.cuda.device_count() == 2

## Configurations

In [4]:
@dataclass
class Config:
#     gemma_dir = '/kaggle/input/gemma-2-9b-it/gemma-2-9b-it-palash-model'
    gemma_dir = '/kaggle/input/gemma-2-9b-hf'
#     gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-8bit/1'
    finetuned_model_dir = '/kaggle/input/gemma2-2stage-checkpoint200-0729'
#     finetuned_model_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2400
    batch_size = 2
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()

# Load & pre-process Data 

In [5]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

In [6]:
def process_text(text: str) -> str:
    ls_str = json.loads(text)
    ls_str = ["" if s is None else s for s in ls_str]
    ls_str = ["null" if s == "" else s for s in ls_str]
    ls_str = [repr(s) for s in ls_str]
#     norm_ls = [unicodedata.normalize('NFKC', s).encode('unicode-escape').decode('utf-8') for s in ls_str]
    ls_str = [unicodedata.normalize('NFKC', s) for s in ls_str]

    return " ".join(ls_str)

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))

,id,prompt,response_a,response_b
0,136060,"'I have three oranges today, I ate an orange y...",'You have two oranges today.','You still have three oranges. Eating an orang...
1,211333,"""You are a mediator in a heated political deba...","""Thank you for sharing the details of the situ...","""Mr Reddy and Ms Blue both have valid points i..."
2,1233961,'How to initialize the classification head whe...,"""When you want to initialize the classificatio...","""To initialize the classification head when pe..."


# Tokenize

In [7]:
def tokenize(tokenizer, df):

    for index, row in df.iterrows():

        # トークン化
        prompt_token = tokenizer.encode("User prompt: " + row.prompt, add_special_tokens=False)
        res_a_token = tokenizer.encode("\n\nModel A :\n" + row.response_a, add_special_tokens=False)
        res_b_token = tokenizer.encode("\n\n--------\n\nModel B:\n" + row.response_b, add_special_tokens=False)


        # 各トークンの最大数と使用トークン数
        prompt_max_token = cfg.max_length // 2
        use_prompt_token_size = min(len(prompt_token), prompt_max_token)

        res_a_max_token = (cfg.max_length - use_prompt_token_size) * len(res_a_token) // (len(res_a_token) + len(res_b_token))
        use_res_a_token_size = min(len(res_a_token), res_a_max_token)

        res_b_max_token = (cfg.max_length - use_prompt_token_size) * len(res_b_token) // (len(res_a_token) + len(res_b_token))
        use_res_b_token_size = min(len(res_b_token), res_b_max_token)


        # すべてが制限を超えていたら終了
        if use_prompt_token_size == prompt_max_token & use_res_a_token_size == res_a_max_token & use_res_b_token_size == res_b_max_token:
            pass

        else:
            while(1):
                # 余っているトークン
                remaining_tokens = cfg.max_length - use_prompt_token_size - use_res_a_token_size - use_res_b_token_size


                # 未確定の列の数
                not_confirm_token = sum((use_prompt_token_size < len(prompt_token), use_res_a_token_size < len(res_a_token), use_res_b_token_size < len(res_b_token)))

                if not_confirm_token == 0:
                    break

                divide_remaining_tokens = remaining_tokens // not_confirm_token
                if divide_remaining_tokens == 0:
                    break

                # 余っているトークン割り当て
                if use_prompt_token_size < len(prompt_token):
                    additional_tokens = min(len(prompt_token) - use_prompt_token_size, divide_remaining_tokens)
                    use_prompt_token_size += additional_tokens
                    remaining_tokens -= additional_tokens

                if use_res_a_token_size < len(res_a_token):
                    additional_tokens = min(len(res_a_token) - use_res_a_token_size, divide_remaining_tokens)
                    use_res_a_token_size += additional_tokens
                    remaining_tokens -= additional_tokens

                if use_res_b_token_size < len(res_b_token):
                    additional_tokens = min(len(res_b_token) - use_res_b_token_size, divide_remaining_tokens)
                    use_res_b_token_size += additional_tokens
                    remaining_tokens -= additional_tokens

        # 更新
        df.loc[index, 'prompt'] = tokenizer.decode(prompt_token[:use_prompt_token_size//2] + prompt_token[-use_prompt_token_size//2:])
        df.loc[index, 'response_a'] = tokenizer.decode(res_a_token[:use_res_a_token_size//2] + res_a_token[-use_res_a_token_size//2:])
        df.loc[index, 'response_b'] = tokenizer.decode(res_b_token[:use_res_b_token_size//2] + res_b_token[-use_res_b_token_size//2:])

        
    # トークン化
    text = [p + r_a + r_b for p, r_a, r_b in zip(df['prompt'], df['response_a'], df['response_b'])]
    tokenized = tokenizer(text, max_length=cfg.max_length, truncation=True, padding=False)
    input_ids = tokenized.input_ids
    attention_mask = tokenized.attention_mask

    return input_ids, attention_mask

In [8]:
%%time

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/gemma2-2stage-0726/gemma2-tokenizer')
# tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test)
data["length"] = data["input_ids"].apply(len)

CPU times: user 832 ms, sys: 127 ms, total: 959 ms
Wall time: 1.08 s


In [9]:
print(tokenizer.decode(data["input_ids"][0]))

<bos>User prompt: 'I have three oranges today, I ate an orange yesterday. How many oranges do I have?'

Model A :
'You have two oranges today.'

--------

Model B:
'You still have three oranges. Eating an orange yesterday does not affect the number of oranges you have today.'<eos>


# Load model

In [10]:
# 8bit量子化を適用
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
)

In [11]:
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
base_model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map=device_0,
    use_cache=False,
    quantization_config=bnb_config,
)
base_model_0.config.pad_token_id = tokenizer.pad_token_id

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
base_model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map=device_1,
    use_cache=False,
    quantization_config=bnb_config,
)

base_model_1.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/gemma-2-9b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/gemma-2-9b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Load LoRA adapter

# Inference


In [12]:
model_0 = PeftModel.from_pretrained(base_model_0, cfg.finetuned_model_dir)
model_1 = PeftModel.from_pretrained(base_model_1, cfg.finetuned_model_dir)

In [13]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df

In [14]:
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")

elapsed time: 4.957403182983398


In [15]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)

,id,winner_model_a,winner_model_b,winner_tie
2,1233961,0.105497,0.689945,0.204558
0,136060,0.001879,0.973305,0.024816
1,211333,0.249666,0.401457,0.348877
